In [1]:
import numpy as np
from sklearn.decomposition import PCA

In [2]:
data = [[-0.07951900675624138, -0.0748285682810908, -0.040703740206813166],
 [0.016328440756408213, 0.004389490395726725, 0.007666520377164143],
 [-0.0062361137515056925, -0.011994705082681426, -0.000811291405063197],
 [0.014682267264510773, 0.011911646270392484, 0.015667328635023542],
 [0.07028169214636204, 0.055305150187513086, 0.035307926421490055],
 [0.10332016952849882, 0.06707424288733213, 0.0303233037983823],
 [-0.027150928580143063, 0.09716146393786146, 0.05350335812335935],
 [-0.03922886558703782, 0.06794381985570258, 0.0451626407944985],
 [0.09362264425185074, 0.09082825768743655, 0.059282029573952316],
 [0.12050823524757637, 0.09574824302747886, 0.041406367311554115]]

In [3]:
model = PCA()

In [4]:
model.fit(data)

PCA()

In [5]:
r1 = model.transform(data)
r1 # results from sklearn PCA model

array([[ 0.16755719, -0.02675022,  0.00502684],
       [ 0.03440636, -0.02237416, -0.00229585],
       [ 0.06351083, -0.02085121, -0.00200449],
       [ 0.02868194, -0.01342914, -0.00553981],
       [-0.04447039, -0.0157479 , -0.00369439],
       [-0.0744747 , -0.03240472,  0.00509147],
       [-0.00305583,  0.08319999,  0.0046895 ],
       [ 0.02596206,  0.06960511, -0.00181098],
       [-0.09034131,  0.0002846 , -0.0081962 ],
       [-0.10777615, -0.02153235,  0.00873389]])

## Serilaise model to ONNX

In [6]:
from skl2onnx import to_onnx

In [7]:
data_for_types = np.array(data)
onx = to_onnx(model, X=data_for_types)

In [8]:
print(f"binary representation:\n{onx.SerializeToString()}")

binary representation:
b'\x08\x08\x12\x08skl2onnx\x1a\x061.10.4"\x07ai.onnx(\x002\x00:\xa1\x02\n!\n\x01X\n\x04mean\x12\nsub_result\x1a\x03Sub"\x03Sub:\x00\n:\n\nsub_result\n\x10transform_matrix\x12\x08variable\x1a\x06MatMul"\x06MatMul:\x00\x12\tONNX(PCA)*b\x08\x03\x08\x03\x10\x0bB\x10transform_matrixRH\x9b\x16\xebH\x0b\xac\xe7\xbf\x8e\x1d\x9e(\xc8~\xe5\xbf+\xeeq4\xdb*\xa4\xbf\xe0\x17\xd2w\x13;\xe3\xbfkT#zWC\xe4?j\xfe\xd5\xb3\xe96\xdf?e\xb9d\x99\r_\xd3\xbf\xcc\xeaq\xd1\x1f\x9b\xd8?\xcc\x8b8\xa8M\xe8\xeb\xbf*$\x08\x03\x10\x0bB\x04meanR\x18\x10\xbf\xa3\x96\xfbL\x9b?\xfa\x12\xa7TD\xa9\xa4?\xd5c4\x95\xd4E\x99?Z\x11\n\x01X\x12\x0c\n\n\x08\x0b\x12\x06\n\x00\n\x02\x08\x03b\x18\n\x08variable\x12\x0c\n\n\x08\x0b\x12\x06\n\x00\n\x02\x08\x03B\x04\n\x00\x10\r'


In [9]:
readble_serialised_onx = repr(onx)
print(f"Human readable: {readble_serialised_onx}") # not sure how to rebuild from this

Human readable: ir_version: 8
producer_name: "skl2onnx"
producer_version: "1.10.4"
domain: "ai.onnx"
model_version: 0
doc_string: ""
graph {
  node {
    input: "X"
    input: "mean"
    output: "sub_result"
    name: "Sub"
    op_type: "Sub"
    domain: ""
  }
  node {
    input: "sub_result"
    input: "transform_matrix"
    output: "variable"
    name: "MatMul"
    op_type: "MatMul"
    domain: ""
  }
  name: "ONNX(PCA)"
  initializer {
    dims: 3
    dims: 3
    data_type: 11
    name: "transform_matrix"
    double_data: -0.7397514747796835
    double_data: -0.6717263024628879
    double_data: -0.03938946738432166
    double_data: -0.6009614315269083
    double_data: 0.6332204232700297
    double_data: 0.48772661745167645
    double_data: -0.30267658214406473
    double_data: 0.3844680352555485
    double_data: -0.8721073422969936
  }
  initializer {
    dims: 3
    data_type: 11
    name: "mean"
    double_data: 0.0266608534520279
    double_data: 0.040353904088567163
    double_

## Apply model (using ONNX runtime)

In [10]:
from onnxruntime import InferenceSession

sess = InferenceSession(onx.SerializeToString())
r2 = sess.run(None, {'X': data_for_types})[0]
r2

array([[ 0.16755719, -0.02675022,  0.00502684],
       [ 0.03440636, -0.02237416, -0.00229585],
       [ 0.06351083, -0.02085121, -0.00200449],
       [ 0.02868194, -0.01342914, -0.00553981],
       [-0.04447039, -0.0157479 , -0.00369439],
       [-0.0744747 , -0.03240472,  0.00509147],
       [-0.00305583,  0.08319999,  0.0046895 ],
       [ 0.02596206,  0.06960511, -0.00181098],
       [-0.09034131,  0.0002846 , -0.0081962 ],
       [-0.10777615, -0.02153235,  0.00873389]])

In [11]:
r1 - r2  # get the  diffs

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])